In [533]:
#packages
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.linalg import qr
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from autograd import grad, hessian
import autograd.numpy as anp
from sklearn.metrics import precision_score, recall_score, f1_score

In [577]:
def sketch_matrix(m, n_columns, non_zero_entries):
    """Generates a sketching matrix S with random ±1 entries."""
    S = np.zeros((m, n_columns))
    scaling_factor = 1 / np.sqrt(non_zero_entries)
    
    for col in range(n_columns):
        nz_positions = np.random.choice(m, non_zero_entries, replace=False)
        values = np.random.choice([1, -1], non_zero_entries) * scaling_factor
        for idx, value in zip(nz_positions, values):
            S[idx, col] = value
    
    return S

def hessian_sqrt(hessian, lam=1e-3):
    """ Computes the square root of the Hessian using Cholesky decomposition with regularization """
    try:
        # Add a small regularization term to the Hessian to ensure it is positive semi-definite
        hessian_regularized = hessian + lam * np.eye(hessian.shape[0])
        
        L = anp.linalg.cholesky(hessian_regularized)  # Perform Cholesky decomposition
        return L
    except anp.linalg.LinAlgError:
        print("Warning: Hessian is not positive semidefinite after regularization.")
        return None


def newton_sketch(f, x0, non_zero_entries, num_iters=100, lambda_identity=1e-3):
    # Autograd gradient computation
    f_grad = grad(f)
    
    # Initialize
    x = x0.copy()
    dim = len(x0)
    
    for _ in range(num_iters):
        grad_val = f_grad(x)  # Compute gradient at current point
        H = hessian_sqrt(x)  # Assume Hessian is passed directly for Cholesky
        if H is None:
            return x
        
        S = sketch_matrix(dim, dim, non_zero_entries)  # Generate sketch matrix
        
        # Compute sketched Hessian approximation
        H_sketch = (S.T @ H.T) @ (S @ H)
        H_sketch += lambda_identity * np.eye(H_sketch.shape[0])
        
        
        # Solve Newton update step
        delta_x = anp.linalg.solve(H_sketch, -grad_val)
        
        # Update x
        x += delta_x
        
    return x

#function to be used
def least_squares_loss(x, A, b):
    """
    Computes the least squares loss: f(x) = ||Ax - b||_2^2
    """
    return anp.sum((A @ x - b) ** 2)

def logistic_loss(x, A, b):
    """
    Computes the logistic loss for binary classification
    """
    z = anp.dot(A, x)  # Use anp.dot instead of @
    return anp.sum(anp.logaddexp(0, -b * z))

In [578]:

# Test function: least squares loss
A = np.array([[1, 2], [3, 4], [5, 6]])  # A simple 3x2 matrix
b = np.array([7, 8, 9])  # Simple vector
x0 = np.array([0.0, 0.0])  # Initial guess for the optimization

# Set non_zero_entries as a value suitable for sketching
non_zero_entries = 2

# Call newton_sketch with least squares loss
result = newton_sketch(lambda x: least_squares_loss(x, A, b), x0, non_zero_entries, num_iters=100)

print("Optimized solution:", result)

Optimized solution: [-3.20075698e+10  9.77599027e+04]


In [579]:
# Function to solve least squares directly using normal equation
def least_squares_normal(A, b):
    return np.linalg.inv(A.T @ A) @ A.T @ b

# Test data
A = np.array([[1, 2], [3, 4], [5, 6]])  # A simple 3x2 matrix
b = np.array([7, 8, 9])  # Simple vector

# Compute the solution using normal equation
x_normal = least_squares_normal(A, b)

print("Optimized solution without Newton (normal equation):", x_normal)

Optimized solution without Newton (normal equation): [-6.   6.5]


In [535]:
df = pd.read_excel("../Dataset/employability.xlsx")

In [536]:
df

,Name of Student,GENERAL APPEARANCE,MANNER OF SPEAKING,PHYSICAL CONDITION,MENTAL ALERTNESS,SELF-CONFIDENCE,ABILITY TO PRESENT IDEAS,COMMUNICATION SKILLS,Student Performance Rating,CLASS
0,Student 1,4,5,4,5,5,5,5,5,Employable
1,Student 2,4,4,4,4,4,4,3,5,Employable
2,Student 3,4,3,3,3,3,3,2,5,LessEmployable
3,Student 4,3,3,3,2,3,3,3,5,LessEmployable
4,Student 5,4,4,3,3,4,4,3,5,Employable
...,...,...,...,...,...,...,...,...,...,...
2977,Student 2996,4,3,3,3,3,3,2,5,Employable
2978,Student 2997,3,4,4,4,4,4,4,5,Employable
2979,Student 2998,4,5,4,5,4,4,4,5,Employable
2980,Student 2999,4,4,4,3,4,4,3,5,LessEmployable


In [537]:
# Drop non-numeric columns (like student names)
df = df.drop(columns=["Name of Student"])

df["CLASS"] = df["CLASS"].map({"Employable": 1, "LessEmployable": 0})

# Separate features (A) and target variable (b)
A = df.drop(columns=["CLASS"]).values  # Convert features to NumPy array
b = df["CLASS"].values  # Target variable
A = A.astype(np.float64)
b = b.astype(np.float64)

In [538]:
A

array([[4., 5., 4., ..., 5., 5., 5.],
       [4., 4., 4., ..., 4., 3., 5.],
       [4., 3., 3., ..., 3., 2., 5.],
       ...,
       [4., 5., 4., ..., 4., 4., 5.],
       [4., 4., 4., ..., 4., 3., 5.],
       [4., 4., 4., ..., 4., 4., 5.]])

In [539]:
b

array([1., 1., 0., ..., 1., 0., 1.])

In [540]:
x0 = np.random.randn(A.shape[1]) #columns = number of features
m = 5  # Number of rows in sketch matrix
non_zero_entries = 2

In [541]:
print(f"A shape: {A.shape}")
print(f"x shape: {x0.shape}")
print(f"b shape: {b.shape}")

A shape: (2982, 8)
x shape: (8,)
b shape: (2982,)


In [542]:
optimized_x, final_lambda = unconstrained_newton_sketch(logistic_loss, x0, m, A, b, non_zero_entries)

Iteration 11


In [543]:
y_pred = 1 / (1 + np.exp(-A @ optimized_x)) 
y_pred = (y_pred > 0.5).astype(int)

In [544]:
precision = precision_score(b, y_pred)
recall = recall_score(b, y_pred)
f1 = f1_score(b, y_pred)
accuracy = np.mean(y_pred == b)


print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Model Accuracy: 57.98%
Precision: 0.57981220657277
Recall: 1.0
F1 Score: 0.7340267459138188


In [545]:
def standard_newton_method(f, x0, A, b, tolerance=1e-4, a=0.1, b_factor=0.5, max_iter=1000):
    grad_f = grad(f)
    hess_f = hessian(f)
    
    xt = x0
    for t in range(max_iter):
        grad_value = grad_f(xt, A, b)
        hess_value = hess_f(xt, A, b)
        
        # Add small regularization to ensure invertibility
        hess_value += 1e-5 * np.eye(hess_value.shape[0])
        
        delta_xt = np.linalg.solve(hess_value, grad_value)
        
        lambda_t = np.dot(grad_value, delta_xt)
        
        if lambda_t**2 / 2 <= tolerance and t > 0:
            print(f"Iteration {t}")
            break
        
        # Backtracking line search
        step_size = 1.0
        while f(xt - step_size * delta_xt, A, b) > f(xt, A, b) - a * step_size * lambda_t:
            step_size *= b_factor
        
        xt = xt - step_size * delta_xt
    
    return xt, np.abs(lambda_t)

In [546]:
# Timing the unconstrained Newton sketch method
start_time_sketch = time.time()
optimized_x_sketch, final_lambda_sketch = unconstrained_newton_sketch(logistic_loss, x0, m, A, b, non_zero_entries)
end_time_sketch = time.time()
execution_time_sketch = end_time_sketch - start_time_sketch

# Predicting using the unconstrained Newton sketch method
y_pred_sketch = 1 / (1 + np.exp(-A @ optimized_x_sketch)) 
y_pred_sketch = (y_pred_sketch > 0.5).astype(int)

# Calculating metrics for unconstrained Newton sketch
accuracy_sketch = np.mean(y_pred_sketch == b)
precision_sketch = precision_score(b, y_pred_sketch)
recall_sketch = recall_score(b, y_pred_sketch)
f1_sketch = f1_score(b, y_pred_sketch)

# Timing the standard Newton method
start_time_standard = time.time()
optimized_x_standard, final_lambda_standard = standard_newton_method(logistic_loss, x0, A, b)
end_time_standard = time.time()
execution_time_standard = end_time_standard - start_time_standard

# Predicting using the standard Newton method
y_pred_standard = 1 / (1 + np.exp(-A @ optimized_x_standard)) 
y_pred_standard = (y_pred_standard > 0.5).astype(int)

# Calculating metrics for standard Newton method
accuracy_standard = np.mean(y_pred_standard == b)
precision_standard = precision_score(b, y_pred_standard)
recall_standard = recall_score(b, y_pred_standard)
f1_standard = f1_score(b, y_pred_standard)

# Displaying the results
print(f"Unconstrained Newton Sketch Method Results:")
print(f"Execution Time: {execution_time_sketch:.4f} seconds")
print(f"Accuracy: {accuracy_sketch * 100:.2f}%")
print(f"Precision: {precision_sketch:.4f}")
print(f"Recall: {recall_sketch:.4f}")
print(f"F1 Score: {f1_sketch:.4f}\n")

print(f"Standard Newton Method Results:")
print(f"Execution Time: {execution_time_standard:.4f} seconds")
print(f"Accuracy: {accuracy_standard * 100:.2f}%")
print(f"Precision: {precision_standard:.4f}")
print(f"Recall: {recall_standard:.4f}")
print(f"F1 Score: {f1_standard:.4f}")


Iteration 9
Iteration 13
Unconstrained Newton Sketch Method Results:
Execution Time: 0.0357 seconds
Accuracy: 58.52%
Precision: 0.5829
Recall: 1.0000
F1 Score: 0.7365

Standard Newton Method Results:
Execution Time: 0.0368 seconds
Accuracy: 57.98%
Precision: 0.5798
Recall: 1.0000
F1 Score: 0.7340
